In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
!pip install mace-torch
!pip install spglib
!pip install nglview==3.0.3
!pip install bvlain

In [3]:
import torch
import numpy as np
import mace

import matplotlib as mpl
import matplotlib.pyplot as plt

from mace.calculators import mace_mp, MACECalculator
from ase.calculators.loggingcalc import LoggingCalculator
from ase.optimize import BFGS

from ase.visualize import view
from ase import build, units, atoms
from ase.io import read, write, Trajectory
from ase.io.animation import write_gif

from ase.constraints import ExpCellFilter, StrainFilter, UnitCellFilter
from ase.spacegroup.symmetrize import FixSymmetry, check_symmetry
from spglib import get_spacegroup

In [ ]:
# Printing True if GPU is connected
print(torch.cuda.is_available())

In [ ]:
# List of generic structures to be imported
from ase.collections import g2
print(g2.names)

In [ ]:
# Building a water molecule
atoms = build.molecule('H2O')
view(atoms, viewer='x3d')

In [ ]:
# Setting the calculator for the atom structure
calculator = mace_mp(model="medium", dispersion=False, default_dtype="float64", device='cuda')
atoms.calc = calculator

print(atoms.get_potential_energy())

In [ ]:
# Setting the MACE-MP-0 Calculator
calculator = MACECalculator(model_paths='/content/drive/MyDrive/Chem_4PB3/Resources/2024-01-07-mace-128-L2_epoch-199.model',
                            dispersion=False, device='cuda', default_dtype='float64')
atoms.calc = calculator

print(atoms.get_potential_energy())

In [ ]:
# Track Data
nsteps = []
energies = []
log_calc = LoggingCalculator(calculator)

print(atoms.symbols)

# Set Log
atoms.calc = log_calc

# Optimise
opt = BFGS(atoms, trajectory='/content/drive/MyDrive/Chem_4PB3/Resources/Optimisation_Stuff/H2O.traj')

In [ ]:
# Run Optimise
opt.run(fmax=1e-4)
print('Finished!!!')

In [ ]:
# Plot Out
plt.figure(figsize=(10,10))
log_calc.plot(markers=['r-', 'b-'], energy=True, lw=2)
plt.show()

In [ ]:
print(atoms.get_potential_energy())

In [ ]:
from ase.vibrations import Vibrations

# Running analysis of the vibrational modes
# of H2O
vib = Vibrations(atoms)
vib.run()
vib.summary()
vib.write_mode(n=None, kT=0.02585199101165164, nimages=60)

In [ ]:
import os

# Set the directory to connect
directory = '/content/'

# Iterate over the listed files in the directory
for filename in os.listdir(directory):
  f = os.path.join(directory, filename)

  if f.endswith('.traj'): # If it is a trajectory file, it will proceed
    traj = Trajectory(f)
    write_gif(f.strip('.traj') + '.mp4', traj, interval=33, rotation=('270x,90y,0z')) #Writing an mp4 file

In [ ]:
from IPython.display import Video
Video('/content/vib.6.mp4', embed=True) # Vibrational mode 6 for H2O

# If the above fails:
# Video('/content/drive/MyDrive/Chem_4PB3/Resources/vib.6.mp4', embed=True)

In [4]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
import nglview as nv

traj = Trajectory('/content/vib.6.traj')
view(traj, viewer='ngl')

In [6]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
# Importing Stuff
atoms = read('/content/drive/MyDrive/Chem_4PB3/Resources/Na4Sn2Ge5O16_fixed.cif')

# Set Calculator
atoms.calc = calculator
print(atoms.symbols)

# Track Data
nsteps = []
energies = []
log_calc = LoggingCalculator(calculator)

# Set Log
atoms.calc = log_calc

# Set Cell filter (preserve unit cell ratioe or symmetry)
# atoms = ExpCellFilter(atoms, hydrostatic_strain=False)
atoms.set_constraint(FixSymmetry(atoms))

view(atoms, viewer='ngl')

In [ ]:
from ase.visualize.plot import plot_atoms

# Begin Plot
fig, axarr = plt.subplots(1, 2, figsize=(10, 7))

plot_atoms(atoms, axarr[0], radii=0.3, rotation=('0x,0y,0z')).set_title('Pristine')


atoms.rattle(1) #Angstom, gaussian


plot_atoms(atoms, axarr[1], radii=0.3, rotation=('0x,0y,0z')).set_title('Rattled')

plt.show()

In [ ]:
print("Cell size before: ", atoms.cell)

In [ ]:
# Run Optimise
opt = BFGS(UnitCellFilter(atoms), trajectory='/content/trajectory.traj')
opt.run(fmax=1e-4)

print("Cell size after : ", atoms.cell)
print("Spacegroup: ", get_spacegroup((atoms.cell, atoms.get_scaled_positions(), atoms.numbers), symprec=1e-2))
atoms.write('/content/optimisation.cif')

# Plot Out
plt.figure(figsize=(10,10))
log_calc.plot(markers=['r-', 'b-'], energy=True, lw=2)
plt.show()

print('Finished!!!')

Example code for running iterations:

```
for i in range(iter):
    print('Iteration: ', i+1)
    # Set Savestate
    trajsave = '/content/Trajectories/Trajectory_1_'
    trajsave += str(i)
    trajsave += '.traj'
    cifsave = '/content/TCIF/Crystal_1_'
    cifsave += str(i)
    cifsave += '.cif'

    #set atoms
    atoms = read('/content/drive/MyDrive/Chem_4PB3/Resources/Na4Sn2Ge5O16_fixed.cif')
    atoms.set_constraint(FixSymmetry(atoms))
    atoms.rattle(stdev=1, seed=i) # seed required to generate different randomness
    atoms.calc = calculator

    # Optimise
    opt = BFGS(UnitCellFilter(atoms), trajectory=trajsave)
    opt.run(fmax=1e-4)
    atoms.write(cifsave)

    # Output Params
    print('\n\n')
    print("Cell size after : ", atoms.cell)
    print("Spacegroup: ", get_spacegroup((atoms.cell, atoms.get_scaled_positions(), atoms.numbers), symprec=1e-2))
    print('Iteration: ', i+1)
```


To create a compilation of trajectories:
```
# Import Trajectories
traj = []
for i in range(15):
    traject = '/content/Trajectories/Trajectory_1_' + str(i) + '.traj'
    traj.append(Trajectory(traject))

# Draft Compiled Trajectory
write('/content/Compiled.traj', '')
for i in range(len(traj[0])):
    atom = None
    atoms = []
    for n in range(len(traj)):
        atoms.append(traj[n][i])
    atom = atoms[0] + atoms[1] + atoms[2] + atoms[3] + atoms[4] + atoms[5] + atoms[6] + atoms[7] +\
            atoms[8] + atoms[9] + atoms[10] + atoms[11] + atoms[12] + atoms[13] + atoms[14]
    with Trajectory('/content/Compiled.traj', mode='a') as trajectory:
        trajectory.write(atom)
```
Annoyingly, it is surprisingly difficult to work with trajectory files containing more than one <code>atoms</code> object!

In [10]:
from google.colab import output
output.enable_custom_widget_manager()

This dataset had one outlier convergence (rattle = 1 A):

In [ ]:
traj = Trajectory('/content/drive/MyDrive/Chem_4PB3/Resources/Compiled_out.traj')
view(traj, viewer='ngl')
# Use atom index colour to show the overlap converging

# write('/content/drive/MyDrive/Chem_4PB3/Resources/Compiled.mp4', traj, interval=33, rotation=('45x,45y,35z'))

In [ ]:
length = []
for i in range(15):
  traj = Trajectory('/content/drive/MyDrive/Chem_4PB3/Resources/Trajectories/Trajectory_1_' + str(i) + '.traj')
  length.append(len(traj))
print(length)
print('Trajectory 13: ', length[13], '\n\n\n')

traj = Trajectory('/content/drive/MyDrive/Chem_4PB3/Resources/Trajectories/Trajectory_1_13.traj')
print('Initial Energy (eV): ', traj[0].get_total_energy())
print('Final Energy (eV): ', traj[-1].get_total_energy())

With the outlier removed:

In [13]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
traj = Trajectory('/content/drive/MyDrive/Chem_4PB3/Resources/Compiled.traj')
view(traj, viewer='ngl')
# Use atom index colour to show the overlap converging

# write('/content/drive/MyDrive/Chem_4PB3/Resources/Compiled.mp4', traj, interval=33, rotation=('45x,45y,35z'))

In [ ]:
# Set energies
energies = []
for i in range(15):
    if i != 13: # Delete Outlier
        traj = Trajectory('/content/drive/MyDrive/Chem_4PB3/Resources/Trajectories/Trajectory_1_' + str(i) + '.traj')
        energy = []
        for n in range(170):
            energy.append(traj[n].get_total_energy())
        energies.append(energy)
        print('Iteration: ', i + 1)

energies = np.array(energies)

# Average by iteration
average = []
stdev = []
for i in range(170):
    average = np.append(average, np.average(energies[:, i]))
    stdev = np.append(stdev, np.std(energies[:, i]))

In [ ]:
# Set linspace for dataset
x1 = np.linspace(1, 170, 170)
x2 = np.linspace(1, 20, 20)
y = average

# Plotting the set
fig, ax = plt.subplots(figsize=(10,10), layout="tight")
ax.plot(x1, average, color='black')
plt.fill_between(x1, y-stdev, y+stdev, color='red', alpha=.3) # This is the standard deviation
ax.set_xlabel('Iteration')
ax.set_ylabel('Average Energy (eV)')
ax.set_title('Average Energy Over Iteration Number', pad=30)

plt.show()

## Zoomed in on first 20 iterations

In [ ]:
fig, ax = plt.subplots(figsize=(10,10), layout="tight")
ax.plot(x2, average[:20], color='black')
# ax.errorbar(np.linspace(1, 170, 170), average, yerr=stdev, fmt='none', color='red')
plt.fill_between(x2, y[:20]-stdev[:20], y[:20]+stdev[:20], color='red', alpha=.3)
ax.set_xlabel('Iteration')
ax.set_xticks(x2)
ax.set_ylabel('Average Energy (eV)')
ax.set_title('Average Energy Over Iteration Number', pad=30)

plt.show()

In [15]:
from bvlain import Lain

# Actually Using: BVlain
# https://pypi.org/project/bvlain/
# https://bvlain.readthedocs.io/en/latest/index.html

# Initialise File
file = '/content/drive/MyDrive/Chem_4PB3/Resources/Optimisation_Stuff/Optimisation_0_3.cif'

# Set Calculator
calc = Lain(verbose=True)

# Set State
st = calc.read_file(file)

params = {'mobile_ion': 'Na1+',    # mobile specie
		  'r_cut': 10.0,           # cutoff for interaction between the mobile species and framework
		  'resolution': 0.1,	   # distance between the grid points
		  'k': 100                 # maximum number of neighbors to be collected for each point
}

## This Crashes:

I don't know why

In [ ]:
# Run Distributions
_ = calc.bvse_distribution(**params)
# _ = calc.void_distribution(**params)

# Perform Percolation Analysis
calc.percolation_barriers(encut = 5.0)

# Create Savestate
savestate = file.replace('.cif', '_bvel')

# Write Grid File
calc.write_grd(filename = savestate, task = 'bvse')  # saves .grd file
calc.write_cube(filename = savestate, task = 'bvse')  # saves .cube file

# Check for Mismatches
table = calc.mismatch(r_cut = 4.0)
# print(table.to_string())
print('Finished!!!')

# MAKE SURE TO SET ISOSURFACE TO 0.4
# MUST SET THE ISOSURFACE TO NEGATIVE

## Displaying the Ion Channels

In [ ]:
from IPython.display import Image
Image('/content/drive/MyDrive/Chem_4PB3/Figures/Na4Sn2Ge5O16.png', embed=True, width='1000')

In [ ]:
Image('/content/drive/MyDrive/Chem_4PB3/Figures/Na4Sn2Ge5O16_NoAtoms.png', embed=True, width='1000')

In [ ]:
Image('/content/drive/MyDrive/Chem_4PB3/Figures/Na4Sn2Ge5O16_NoPolyhedra.png', embed=True, width='1000')